Source: http://blog.conceptnet.io/posts/2017/how-to-make-a-racist-ai-without-really-trying/

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

In [2]:
TOKEN_RE = re.compile(r"\w.*?\b")

In [3]:
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

def load_embeddings(filename):
    
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        
        # Get the file
        for i, line in enumerate(infile):
            if i % 100000 == 0:
                print(i)
            items = line.rstrip().split(' ')
            if len(items) == 2:
                continue
            labels.append(items[0])
            
            values = np.array([float(x) for x in items[1:]], 'f')
            
            # Normalize the values for geometry calculations
            values = normalize(values.reshape(1, -1))
            
            rows.append(values)
        
        arr = np.vstack(rows)
        return pd.DataFrame(arr, index=labels, dtype='f')

def vecs_to_sentiment(vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(words):
    vecs = embeddings.loc[words].dropna()
    log_odds = vecs_to_sentiment(vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)

def text_to_sentiment(text):
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    sentiments = words_to_sentiment(tokens)
    return sentiments['sentiment'].mean()


In [4]:
embeddings = load_embeddings('data/glove.6B.100d.txt')

0
100000
200000
300000


In [5]:
pos_words = load_lexicon('data/positive-words.txt')
neg_words = load_lexicon('data/negative-words.txt')

In [6]:
pos_vectors = embeddings.reindex(index=pos_words).dropna()
neg_vectors = embeddings.reindex(index=neg_words).dropna()

In [7]:
vectors = pd.concat([pos_vectors, neg_vectors])
targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
labels = list(pos_vectors.index) + list(neg_vectors.index)

In [8]:
train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
    train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)

In [9]:
model = SGDClassifier(loss='log', random_state=0)
model.fit(train_vectors, train_targets)

SGDClassifier(loss='log', random_state=0)

In [10]:
accuracy_score(model.predict(test_vectors), test_targets)

0.8990384615384616

In [11]:
words_to_sentiment(['american','mexican'])

,sentiment
american,1.372401
mexican,-0.333856


In [12]:
words_to_sentiment(test_labels).iloc[:20]

,sentiment
cunts,-1.625684
phobic,-1.174488
narrower,-1.527412
discordance,-2.941351
sourly,-3.185166
enthusiasm,4.436728
like,0.659964
averse,-3.530386
triumphantly,1.407724
auspicious,3.192836


In [13]:
words_to_sentiment(test_labels).iloc[-20:]

,sentiment
disadvantages,-0.884199
brilliance,3.194220
skillfully,1.705191
ire,-3.704145
modern,4.651355
obscured,-3.478131
break-ups,-1.573020
unfounded,-6.031693
scrambling,-2.435312
unexpected,-0.776431
